In [158]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import layers
import tensorflow as tf
import time

In [159]:
tf.__version__

'2.11.0'

In [160]:
from tensorflow.keras.utils import to_categorical as labelEncoding 

T = 15 # terminus_length

X1 = np.load('bpf-740.npy')


X1 = X1[:376,:T,:]


Y  = [1 for _ in range(376)]

Y = labelEncoding(Y, dtype=int)

print(X1.shape)
print(Y.shape)

(376, 15, 20)
(376, 2)


In [161]:
print(X1.shape)
print(X1[0].shape)
print(X1[0])

BUFFER_SIZE = 1000
BATCH_SIZE = 32

train_dataset = tf.data.Dataset.from_tensor_slices(X1).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

(376, 15, 20)
(15, 20)
[[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]]


In [162]:
# Deep Neural Networks:
import tensorflow as tf; print('We\'re using TF-{}.'.format(tf.__version__))
# import keras; print('We\'re using Keras-{}.'.format(keras.__version__))
from tensorflow.keras.layers import (Input, Dense, Dropout, Flatten, BatchNormalization,
                                     Conv1D, Conv2D, MaxPooling1D, MaxPooling2D,
                                     LSTM, GRU, Embedding, Bidirectional, Concatenate)
from tensorflow.keras.regularizers import (l1, l2, l1_l2)
from tensorflow.keras.optimizers import (RMSprop, Adam, SGD)
from tensorflow.keras.models import (Sequential, Model)

# Core:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import interp
import matplotlib.patches as patches

# Performance:
from sklearn.metrics import (confusion_matrix, classification_report, matthews_corrcoef, precision_score, roc_curve, auc)
from sklearn.model_selection import (StratifiedKFold, KFold, train_test_split)

#Utilities:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical as labelEncoding   # Usages: Y = labelEncoding(Y, dtype=int)
from tensorflow.keras.utils import plot_model   

We're using TF-2.11.0.


In [163]:
def build_discriminator():
    ### Head-1:
    input1 = Input(shape=X1[0].shape)

    x = Conv1D(filters=10, kernel_size=4, padding='same', activation='relu', kernel_regularizer=l2(l=0.01))(input1)
    x = BatchNormalization()(x)
    x = Dropout(rate=0.80)(x)

    x = Conv1D(filters=8, kernel_size=3, padding='same', activation='relu', kernel_regularizer=l2(l=0.01))(x)
    x = BatchNormalization()(x)
    x = Dropout(rate=0.70)(x)

    head1 = Flatten()(x)

    # merge
    merge = Concatenate()([head1])

    output = Dense(units=8, activation='relu', kernel_regularizer=l2(l=0.01))(merge)
    output = BatchNormalization()(output)
    output = Dropout(rate=0.70)(output)

    output = Dense(units=2, activation='softmax')(output)

    return Model(inputs=[input1], outputs=[output])

In [164]:
from Bio.Seq import Seq
from Bio.SeqUtils import ProtParam
from Bio.Align import substitution_matrices
from Bio.SubsMat import MatrixInfo


# Function to convert binary profile feature to amino acid sequence
def bpf_to_sequence(binary_profile_feature):
    amino_acids = "ARNDCQEGHILKMFPSTWYVBZX"
    sequence = ""
    for row in binary_profile_feature:
        index = row.argmax()
        sequence += amino_acids[index]
    return sequence

def seq_to_blosum(sequence):
    blosum62 = MatrixInfo.blosum62
    sequence = sequence.upper()
    length = len(sequence)
    blosum_matrix = []
    for aa1 in sequence:
        row = []
        for aa2 in sequence:
            if (aa1, aa2) in blosum62:
                row.append(blosum62[(aa1, aa2)])
            else:
                row.append(blosum62[(aa2, aa1)])
        blosum_matrix.append(row)
    return blosum_matrix

In [165]:
def bpf_to_sequence(binary_profile_feature):
    amino_acids = "ARNDCQEGHILKMFPSTWYVBZX"
    sequence = ""
    for row in binary_profile_feature:
        i = 0
        for item in row:
            i += 1
            if item.numpy() == 1:
                sequence += amino_acids[i]
    return sequence

def seq_to_blosum(sequence):
    blosum62 = MatrixInfo.blosum62
    sequence = sequence.upper()
    length = len(sequence)
    blosum_matrix = []
    for aa1 in sequence:
        row = []
        for aa2 in sequence:
            if (aa1, aa2) in blosum62:
                row.append(blosum62[(aa1, aa2)])
            else:
                row.append(blosum62[(aa2, aa1)])
        blosum_matrix.append(row[:20])
    return blosum_matrix

def bpf_to_blosum_layer(seqs_bpf):
    seqs_blosum = []
    for bpf in seqs_bpf:
        str_seq = bpf_to_sequence(bpf)
        seqs_blosum.append(seq_to_blosum(str_seq))

    print(seqs_blosum)
    return seqs_blosum

def build_generator(latent_dim, output_shape):
    model = Sequential([
        layers.Input(shape=(latent_dim,)),
        layers.Dense(128, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dense(output_shape[0] * output_shape[1], activation='sigmoid'),
        layers.Reshape(output_shape),
        layers.Lambda(lambda x: tf.one_hot(tf.argmax(x, axis=-1), depth=output_shape[1])),
        layers.Lambda(lambda x: tf.cast(x, dtype=tf.int32)),
    ])

    return model

In [166]:
# Define the dimensions
latent_dim = 100  # Dimensionality of the latent space
batch_size = 5
output_shape = (25, 20)  # Desired output shape of the matrix

generator = build_generator(latent_dim, output_shape)

noise = tf.random.normal([batch_size, latent_dim])

generated_matrix = generator(noise, training=False)

generated_matrix

<tf.Tensor: shape=(5, 25, 20), dtype=int32, numpy=
array([[[0, 0, 0, ..., 1, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 1],
        ...,
        [0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 1, 0, 0],
        [1, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 1, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 1, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 1, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]

In [167]:
discriminator = build_discriminator()
# discriminator.load_weights('./acp_mhcnn_weights.h5')

In [168]:
prediction = discriminator.predict([X1[:5,:,:]])
prediction

1/1 [==============================] - 0s 100ms/step


array([[0.4495534 , 0.55044657],
       [0.49329898, 0.50670105],
       [0.48932347, 0.51067656],
       [0.48096982, 0.5190302 ],
       [0.4742102 , 0.5257898 ]], dtype=float32)

In [169]:
noise = tf.random.normal([5, 100])
fake_seqs = generator(noise, training=False)

prediction = discriminator.predict([fake_seqs[:,:15,:]])
print(prediction)

1/1 [==============================] - 0s 109ms/step
[[0.4626859  0.5373141 ]
 [0.49795172 0.5020483 ]
 [0.4081326  0.5918674 ]
 [0.4517066  0.5482935 ]
 [0.500225   0.49977496]]


In [229]:
def discriminator_loss(real_output, fake_output):
    real_loss = tf.keras.losses.BinaryCrossentropy()(tf.ones_like(real_output), real_output)
    fake_loss = tf.keras.losses.BinaryCrossentropy()(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [259]:
def generator_loss(fake_output):
    return tf.keras.losses.BinaryCrossentropy()(tf.ones_like(fake_output), fake_output)

# def generator_loss(disc_output):
#     batch_size = tf.shape(disc_output)[0]
#     num_classes = tf.shape(disc_output)[1]
#     desired_output = tf.concat([tf.zeros((batch_size, 1)), tf.ones((batch_size, num_classes - 1))], axis=1)
    
#     # Define binary cross entropy loss
#     bce = tf.keras.losses.BinaryCrossentropy(from_logits=False)
    
#     # Calculate loss
#     loss = bce(desired_output, disc_output)
    
#     return loss

In [260]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [261]:
noise = tf.random.normal([5, 100])
fake_seqs = generator(noise, training=False)

fake_prediction = discriminator.predict([fake_seqs[:,:15,:]])
real_prediction = discriminator.predict([X1[:5]])
print(fake_prediction)
print(real_prediction)

# disc_loss = discriminator_loss(Y[:5], real_prediction)
gen_loss = generator_loss(fake_prediction)
print(gen_loss)
disc_loss = discriminator_loss(real_prediction, fake_prediction)
print(disc_loss)

1/1 [==============================] - 0s 19ms/step


1/1 [==============================] - 0s 78ms/step
[[0.49848107 0.5015189 ]
 [0.4594654  0.5405347 ]
 [0.49525827 0.5047417 ]
 [0.46043888 0.53956115]
 [0.46481648 0.53518355]]
[[0.50247216 0.49752787]
 [0.52973485 0.47026518]
 [0.54941237 0.4505877 ]
 [0.5500998  0.44990015]
 [0.5424175  0.45758247]]
tf.Tensor(0.6949406, shape=(), dtype=float32)
tf.Tensor(1.3911572, shape=(), dtype=float32)


In [262]:
import os

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [263]:
def generate_and_save_images(model, epoch, test_input):
    pass

In [264]:
BUFFER_SIZE = 1000
BATCH_SIZE = 32

EPOCHS = 50
noise_dim = 100
num_examples_to_generate = 16

# You will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [267]:
@tf.function
def train_step(peptides):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_bpf = generator(noise, training=True)

      real_output = discriminator(peptides, training=True)
      fake_output = discriminator(generated_bpf[:,:15,:], training=True)

      print(real_output)
      print(fake_output)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)
      print(gen_loss)
      print(disc_loss)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    print(gradients_of_generator)
    print(gradients_of_discriminator)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()

    for seq_batch in dataset:
      train_step(seq_batch)

    # Produce images for the GIF as you go
    display.clear_output(wait=True)
    generate_and_save_images(generator,
                             epoch + 1,
                             seed)

    # Save the model every 15 epochs
    if (epoch + 1) % 15 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

  # Generate after the final epoch
  display.clear_output(wait=True)
  generate_and_save_images(generator,
                           epochs,
                           seed)

In [268]:
train(train_dataset, EPOCHS)

Tensor("model_4/dense_27/Softmax:0", shape=(32, 2), dtype=float32)
Tensor("model_4/dense_27/Softmax_1:0", shape=(32, 2), dtype=float32)
Tensor("binary_crossentropy/weighted_loss/value:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)
[None, None, None, None, None, None]
[<tf.Tensor 'AddN_21:0' shape=(4, 20, 10) dtype=float32>, <tf.Tensor 'AddN_22:0' shape=(10,) dtype=float32>, <tf.Tensor 'AddN_23:0' shape=(10,) dtype=float32>, <tf.Tensor 'AddN_24:0' shape=(10,) dtype=float32>, <tf.Tensor 'AddN_25:0' shape=(3, 10, 8) dtype=float32>, <tf.Tensor 'AddN_26:0' shape=(8,) dtype=float32>, <tf.Tensor 'AddN_27:0' shape=(8,) dtype=float32>, <tf.Tensor 'AddN_28:0' shape=(8,) dtype=float32>, <tf.Tensor 'AddN_29:0' shape=(120, 8) dtype=float32>, <tf.Tensor 'AddN_30:0' shape=(8,) dtype=float32>, <tf.Tensor 'AddN_31:0' shape=(8,) dtype=float32>, <tf.Tensor 'AddN_32:0' shape=(8,) dtype=float32>, <tf.Tensor 'AddN_33:0' shape=(8, 2) dtype=float32>, <tf.Tensor 'AddN_34:0' shape=(2,) dt

ValueError: in user code:

    File "/tmp/ipykernel_86276/324403233.py", line 25, in train_step  *
        generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    File "/home/homayoun/anaconda3/envs/acp-proj/lib/python3.10/site-packages/keras/optimizers/optimizer_experimental/optimizer.py", line 1139, in apply_gradients  **
        grads_and_vars = self.aggregate_gradients(grads_and_vars)
    File "/home/homayoun/anaconda3/envs/acp-proj/lib/python3.10/site-packages/keras/optimizers/optimizer_experimental/optimizer.py", line 1105, in aggregate_gradients
        return optimizer_utils.all_reduce_sum_gradients(grads_and_vars)
    File "/home/homayoun/anaconda3/envs/acp-proj/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/utils.py", line 33, in all_reduce_sum_gradients
        filtered_grads_and_vars = filter_empty_gradients(grads_and_vars)
    File "/home/homayoun/anaconda3/envs/acp-proj/lib/python3.10/site-packages/keras/optimizers/o